In [1]:
#####Rank logit model#####
import numpy as np
import pandas as pd
import matplotlib.pyplot  as plt
import numpy.matlib
import scipy.linalg
import itertools
import seaborn as sns
from scipy import sparse
from scipy.stats import norm
from pandas.tools.plotting import scatter_matrix
from numpy.random import *
from scipy import optimize

#np.random.seed(98537)

In [2]:
##多項分布の乱数を生成する関数
def rmnom(pr, n, k, no, pattern):
    z_id = np.argmax((np.cumsum(pr, axis=1) >= np.random.uniform(0, 1, n)[:, np.newaxis]), axis=1)
    if pattern==1:
        Z = sparse.coo_matrix((np.repeat(1, n), (no, np.array(z_id))), shape=(n, k))   #スパース行列の設定
        return z_id, Z
    return z_id

In [3]:
##データの設定
r = 3   #3位まで選択
n = 25000   #ユーザー数
member = 10   #選択可能数
N = member*n   #総レコード数

In [4]:
##idとインデックスの設定
#idの設定
user_id = np.repeat(np.arange(n), np.repeat(member, n))
member_id = np.tile(np.arange(member), n)

#インデックスの設定
user_list = [i for i in range(n)]
member_list = [j for j in range(member)]
for i in range(n):
    user_list[i] = np.array(np.where(user_id==i)[0], dtype="int")
for j in range(member):
    member_list[j] = np.array(np.where(member_id==j)[0], dtype="int")

In [5]:
##説明変数の生成
#衣装タイプの生成
types = 9   #衣装数
CLOTH = np.zeros((N, types), dtype="int")
Prob = np.random.dirichlet(np.repeat(2.0, types), 1).reshape(-1)
for j in range(member):
    CLOTH[member_list[j], ] = np.random.multinomial(1, Prob, n)
CLOTH = CLOTH[:, np.delete(np.arange(types), np.argmin(np.sum(CLOTH, axis=0)))]

#メンバーのshare of voiceを生成
alpha = np.abs(np.random.normal(0, 0.5, member))
AWARE = np.random.dirichlet(alpha, n).reshape(-1)

#メンバーの接触確率を生成
alpha = np.abs(np.random.normal(0, 0.5, member))
CONTACT = np.random.dirichlet(alpha, n).reshape(-1)

#登録からの経過年数
REGIST = np.abs(np.random.normal(0, 1.0, n))

#プレイ時間の対数
PLAY = np.abs(np.random.normal(0, 1.0, n))

#データの結合
x1 = np.hstack((np.repeat(1, n)[:, np.newaxis], REGIST[:, np.newaxis], PLAY[:, np.newaxis]))
x2 = np.hstack((CLOTH, AWARE[:, np.newaxis], CONTACT[:, np.newaxis]))
k1 = x1.shape[1]
k2 = x2.shape[1]

In [10]:
##応答変数の生成
rp = 0
while True:
    rp = rp + 1
    print(rp)
    
    #パラメータの設定
    beta01 = np.random.multivariate_normal(np.repeat(0.75, member-1), np.diag(np.repeat(1.5, member-1)), 1).reshape(-1)
    beta02 = np.random.multivariate_normal(np.repeat(0.0, member-1), np.diag(np.repeat(0.25, member-1)), 1).reshape(-1)
    beta03 = np.random.multivariate_normal(np.repeat(0.0, member-1), np.diag(np.repeat(0.25, member-1)), 1).reshape(-1)
    beta11 = np.random.multivariate_normal(np.repeat(0.0, types-1), np.diag(np.repeat(1.0, types-1)), 1).reshape(-1)
    beta12 = np.array([0.75])
    beta13 = np.array([1.0])
    beta1 = np.vstack((beta01, beta02, beta03))
    beta2 = np.concatenate((beta11, beta12, beta13))
    betat1 = beta1.copy(); betat2 = beta2.copy() 

    #潜在効用を定義
    Util1 = np.hstack((np.dot(x1, beta1), np.repeat(0, n)[:, np.newaxis]))
    Util2 = np.dot(x2, beta2).reshape(n, member)
    Util = np.exp(Util1 + Util2)

    #ランキングを生成
    rank = np.zeros((n, r), dtype="int")
    Rank = np.zeros((n, member, r), dtype="int")
    Prob = np.zeros((n, member, r))
    Z = np.full((n, member, r), 1)

    for j in range(r):
        #選択済みメンバーのflagを削除
        if j >= 1:
            Z[:, :, j] = Z[:, :, j-1] * (1-Rank[:, :, j-1])

        #順位ごとの選択確率を定義
        U = Util * Z[:, :, j]
        Prob[:, :, j] = U / np.sum(U, axis=1)[:, np.newaxis]

        #多項分布から順位を生成
        res = rmnom(Prob[:, :, j], n, member, np.arange(n), 1)
        Rank[:, :, j] = np.array(res[1].todense(), dtype="int")
        rank[:, j] = np.array(res[0], dtype="int")
        
    #break条件
    if np.median(np.max(Prob[:, :, 0], axis=1)) > 0.55:
        break

1
2
3
4
5
